In [1]:
from zipline.pipeline import Pipeline
from zipline.component.research import run_pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import SimpleMovingAverage

## Filters

A Filter is a function from an asset and a moment in time to a boolean:
```
F(asset, timestamp) -> boolean
```
In Pipeline, [Filters](https://www.quantopian.com/help#quantopian_pipeline_filters_Filter) are used for narrowing down the set of securities included in a computation or in the final output of a pipeline. There are two common ways to create a `Filter`: comparison operators and `Factor`/`Classifier` methods.

###Comparison Operators
Comparison operators on `Factors` and `Classifiers` produce Filters. Since we haven't looked at `Classifiers` yet, let's stick to examples using `Factors`. The following example produces a filter that returns `True` whenever the latest close price is above $20.

In [2]:
last_close_price = USEquityPricing.close.latest
close_price_filter = last_close_price > 20

And this example produces a filter that returns True whenever the 10-day mean is below the 30-day mean.

In [3]:
mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)
mean_crossover_filter = mean_close_10 < mean_close_30

Remember, each security will get its own `True` or `False` value each day.

### Factor/Classifier Methods
Various methods of the `Factor` and `Classifier` classes return `Filters`. Again, since we haven't yet looked at `Classifiers`, let's stick to `Factor` methods for now (we'll look at `Classifier` methods later). The `Factor.top(n)` method produces a `Filter` that returns `True` for the top `n` securities of a given `Factor`. The following example produces a filter that returns `True` for exactly 200 securities every day, indicating that those securities were in the top 200 by last close price across all known securities.

In [4]:
last_close_price = USEquityPricing.close.latest
top_close_price_filter = last_close_price.top(200)

For a full list of `Factor` methods that return `Filters`, see [this link](https://www.quantopian.com/help#quantopian_pipeline_factors_Factor).

For a full list of `Classifier` methods that return `Filters`, see [this link](https://www.quantopian.com/help#quantopian_pipeline_classifiers_Classifier).

##Dollar Volume Filter
As a starting example, let's create a filter that returns `True` if a security's 30-day average dollar volume is above $10,000,000. To do this, we'll first need to create an `AverageDollarVolume` factor to compute the 30-day average dollar volume. Let's include the built-in `AverageDollarVolume` factor in our imports:

In [5]:
from zipline.pipeline.factors import AverageDollarVolume

And then, let's instantiate our average dollar volume factor.

In [6]:
dollar_volume = AverageDollarVolume(window_length=30)

By default, `AverageDollarVolume` uses `USEquityPricing.close` and `USEquityPricing.volume` as its `inputs`, so we don't specify them.

Now that we have a dollar volume factor, we can create a filter with a boolean expression. The following line creates a filter returning `True` for securities with a `dollar_volume` greater than 10,000,000:

In [7]:
high_dollar_volume = (dollar_volume > 10000000)

To see what this filter looks like, let's can add it as a column to the pipeline we defined in the previous lesson.

In [8]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30
    
    dollar_volume = AverageDollarVolume(window_length=30)
    high_dollar_volume = (dollar_volume > 10000000)

    return Pipeline(
        columns={
            'percent_difference': percent_difference,
            'high_dollar_volume': high_dollar_volume
        }
    )

If we make and run our pipeline, we now have a column `high_dollar_volume` with a boolean value corresponding to the result of the expression for each security.

In [9]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result

high_dollar_volume  \
2015-05-05 00:00:00+00:00 Equity(000001 [平安银行])                True   
                          Equity(000002 [万 科Ａ])                True   
                          Equity(000004 [国农科技])                True   
                          Equity(000005 [世纪星源])               False   
                          Equity(000006 [深振业Ａ])                True   
                          Equity(000007 [全新好])                 True   
                          Equity(000008 [神州高铁])               False   
                          Equity(000009 [中国宝安])                True   
                          Equity(000010 [美丽生态])               False   
                          Equity(000011 [深物业A])                True   
                          Equity(000012 [南 玻Ａ])                True   
                          Equity(000014 [沙河股份])                True   
                          Equity(000016 [深康佳Ａ])                True   
                          Equity(000017 [深中华A])                True   
                          Equity(000018 [神州长城])                True   
                          Equity(000019 [深深宝Ａ])                True   
                          Equity(000020 [深华发Ａ])                True   
                          Equity(000021 [深科技])                 True   
                          Equity(000022 [深赤湾Ａ])                True   
                          Equity(000023 [深天地Ａ])                True   
                          Equity(000024 [招商地产])                True   
                          Equity(000025 [特 力Ａ])                True   
                          Equity(000026 [飞亚达Ａ])                True   
                          Equity(000027 [深圳能源])                True   
                          Equity(000028 [国药一致])                True   
                          Equity(000029 [深深房Ａ])                True   
                          Equity(000030 [富奥股份])                True   
                          Equity(000031 [中粮地产])                True   
                          Equity(000032 [深桑达Ａ])                True   
                          Equity(000033 [新都退])                False   
...                                                             ...   
                          Equity(603609 [禾丰牧业])                True   
                          Equity(603611 [诺力股份])                True   
                          Equity(603618 [杭电股份])                True   
                          Equity(603636 [南威软件])                True   
                          Equity(603678 [火炬电子])                True   
                          Equity(603686 [龙马环卫])                True   
                          Equity(603688 [石英股份])                True   
                          Equity(603698 [航天工程])                True   
                          Equity(603699 [纽威股份])                True   
                          Equity(603703 [盛洋科技])               False   
                          Equity(603729 [龙韵股份])                True   
                          Equity(603766 [隆鑫通用])                True   
                          Equity(603788 [宁波高发])                True   
                          Equity(603789 [星光农机])               False   
                          Equity(603799 [华友钴业])                True   
                          Equity(603806 [福斯特])                 True   
                          Equity(603808 [歌力思])                 True   
                          Equity(603818 [曲美家居])               False   
                          Equity(603828 [柯利达])                 True   
                          Equity(603869 [北部湾旅])                True   
                          Equity(603883 [老百姓])                False   
                          Equity(603889 [新澳股份])                True   
                          Equity(603898 [好莱客])                 True   
                          Equity(603899 [晨光文具])                True   
                          Equity(603939 [益丰药房])                True   
  

## Applying a Screen
By default, a pipeline produces computed values each day for every asset in the Quantopian database. Very often however, we only care about a subset of securities that meet specific criteria (for example, we might only care about securities that have enough daily trading volume to fill our orders quickly). We can tell our Pipeline to ignore securities for which a filter produces `False` by passing that filter to our Pipeline via the `screen` keyword.

To screen our pipeline output for securities with a 30-day average dollar volume greater than $10,000,000, we can simply pass our `high_dollar_volume` filter as the `screen` argument. This is what our `make_pipeline` function now looks like:

In [10]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30

    dollar_volume = AverageDollarVolume(window_length=30)
    high_dollar_volume = dollar_volume > 10000000

    return Pipeline(
        columns={
            'percent_difference': percent_difference
        },
        screen=high_dollar_volume
    )

When we run this, the pipeline output only includes securities that pass the `high_dollar_volume` filter on a given day. For example, running this pipeline on May 5th, 2015 results in an output for ~2,100 securities

In [11]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
print('Number of securities that passed the filter: %d' % len(result))
result

Number of securities that passed the filter: 2511


percent_difference
2015-05-05 00:00:00+00:00 Equity(000001 [平安银行])            0.105145
                          Equity(000002 [万 科Ａ])            0.035091
                          Equity(000004 [国农科技])            0.016754
                          Equity(000006 [深振业Ａ])            0.135837
                          Equity(000007 [全新好])             0.116155
                          Equity(000009 [中国宝安])            0.100745
                          Equity(000011 [深物业A])            0.086561
                          Equity(000012 [南 玻Ａ])            0.135719
                          Equity(000014 [沙河股份])            0.008347
                          Equity(000016 [深康佳Ａ])            0.170770
                          Equity(000017 [深中华A])            0.178577
                          Equity(000018 [神州长城])            0.082513
                          Equity(000019 [深深宝Ａ])            0.106169
                          Equity(000020 [深华发Ａ])            0.074627
                          Equity(000021 [深科技])             0.074533
                          Equity(000022 [深赤湾Ａ])            0.075639
                          Equity(000023 [深天地Ａ])           -0.012463
                          Equity(000024 [招商地产])            0.021478
                          Equity(000025 [特 力Ａ])           -0.030485
                          Equity(000026 [飞亚达Ａ])            0.046182
                          Equity(000027 [深圳能源])            0.048828
                          Equity(000028 [国药一致])            0.061024
                          Equity(000029 [深深房Ａ])            0.058638
                          Equity(000030 [富奥股份])            0.097110
                          Equity(000031 [中粮地产])            0.093370
                          Equity(000032 [深桑达Ａ])            0.144914
                          Equity(000035 [中国天楹])           -0.002064
                          Equity(000036 [华联控股])            0.114947
                          Equity(000037 [深南电A])            0.108079
                          Equity(000039 [中集集团])            0.075515
...                                                             ...
                          Equity(603588 [高能环境])            0.055432
                          Equity(603600 [永艺股份])            0.178892
                          Equity(603601 [再升科技])           -0.020769
                          Equity(603606 [东方电缆])            0.116981
                          Equity(603609 [禾丰牧业])            0.064909
                          Equity(603611 [诺力股份])            0.018747
                          Equity(603618 [杭电股份])            0.043706
                          Equity(603636 [南威软件])            0.043761
                          Equity(603678 [火炬电子])            0.136825
                          Equity(603686 [龙马环卫])            0.319734
                          Equity(603688 [石英股份])            0.043137
                          Equity(603698 [航天工程])            0.044468
                          Equity(603699 [纽威股份])            0.038250
                          Equity(603729 [龙韵股份])            0.136026
                          Equity(603766 [隆鑫通用])            0.093572
                          Equity(603788 [宁波高发])            0.065270
                          Equity(603799 [华友钴业])            0.091847
                          Equity(603806 [福斯特])             0.120451
                          Equity(603808 [歌力思])             0.000000
                          Equity(603828 [柯利达])             0.041188
                          Equity(603869 [北部湾旅])            0.378968
                          Equity(603889 [新澳股份])            0.061421
                          Equity(603898 [好莱客])             0.017080
                          Equity(603899 [晨光文具])            0.041762
                          Equity(603939 [益丰药房])            0.138342
                          Equity(603969 [银龙股份])            0.094364
                          Equity(603988 [中电电机])            0.048184
                          Equity(6039

## Inverting a Filter
The `~` operator is used to invert a filter, swapping all `True` values with `Falses` and vice-versa. For example, we can write the following to filter for low dollar volume securities:

In [12]:
low_dollar_volume = ~high_dollar_volume

This will return `True` for all securities with an average dollar volume below or equal to $10,000,000 over the last 30 days.

In the next lesson, we will look at combining filters.